In [6]:
%%capture

%run ../path_manager.ipynb

In [8]:
import pandas as pd
import glob

# Ngram process

- Generate initial ngram candidates per corpus.
- Remove ngrams in whitelist from uncurated ngram candidates.
- Build a unified curation sheet containing only items not yet curated.
- Store in master ngram list.

In [14]:
master_ngram_whitelist = pd.read_csv('../whitelists/whitelist_ngrams.csv', header=None)
master_ngram_whitelist = master_ngram_whitelist.rename(columns={0: 'ngram'})

final_ngrams = set(master_ngram_whitelist.ngram)

master_ngram_whitelist.head()

,ngram
0,food_and_agriculture_organization
1,international_atomic_energy_agency
2,international_civil_aviation_organization
3,international_civil_service_commission
4,international_fund_for_agricultural_development


In [10]:
CORPUS_ID = 'WB'
ngram_files = glob.glob(os.path.join(get_corpus_path(CORPUS_ID), f'{CORPUS_ID.lower()}_ngrams*.csv'))

In [11]:
updated_unlabelled = pd.DataFrame(columns=['ngram', 'occ'])

new_ngrams = set()

for fname in ngram_files:
    ngrams_df = pd.read_csv(fname)
    if 'keep' in ngrams_df.columns:
        labelled = ngrams_df[ngrams_df.keep == 'y']
        new_ngrams.update(set(labelled.ngram).difference(final_ngrams))
        # dropped = ngrams_df[ngrams_df.keep == 'n']

        unlabelled = ngrams_df[~ngrams_df.keep.isin(['y', 'n'])]
    else:
        unlabelled = ngrams_df[~ngrams_df.ngram.isin(set(final_ngrams).union(new_ngrams))]
        
    if updated_unlabelled.empty:
        updated_unlabelled = unlabelled
    else:
        updated_unlabelled = pd.concat([updated_unlabelled, unlabelled])

updated_unlabelled = updated_unlabelled.groupby('ngram')['occ'].sum().reset_index('ngram').sort_values('occ', ascending=False)

[]